In [1]:
import h5py
import json
import random
import pandas as pd
import sys
import os
import matplotlib.pyplot as plt
from wandb.keras import WandbCallback
import pydicom
sys.path.append('./src')
import numpy as np
#from generators import DataGenerator
from generator_spread import DataGenerator
from models_spread import dota_energies
from preprocessing import DataRescaler
from preprocessing_hamdi import get_scaling_factors
from tensorflow_addons.optimizers import LAMB
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.config import list_physical_devices
import json
import tensorflow as tf
print(list_physical_devices('GPU'))
from models import dota_energies
from preprocessing import DataRescaler
from evaluation import gamma_analysis, error_analysis
from tqdm import tqdm

2024-03-15 06:33:06.336396: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-15 06:33:06.576008: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-15 06:33:07.580061: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-03-15 06:33:07.580148: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU')]


/home/hamdielsayed/.conda/envs/dota_electron/lib/python3.9/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [18]:
ikey='GeometryAll'
Errors=[]
for ID in tqdm(data_df['cropped_geometry_name']):
    try:
        pydicom.dcmread(os.path.join(path,ikey,ID+'.dcm')).pixel_array
    except:
        print(ID)
        Errors.append(ID)

  0%|          | 0/34224 [00:00<?, ?it/s]

In [22]:
data_df = pd.read_pickle(os.path.join(path, 'Filtered_Simulation_Data.pkl'))
data_df_sample = data_df.sample(500, ord=1)


TypeError: sample() got an unexpected keyword argument 'ord'

In [2]:
import os
import pandas as pd

folder_path = r"/tudelft.net/staff-umbrella/simelectrons/OneGeometryMultipleEnergyMultipleSpread"  # Replace with the actual folder path

# Get a list of all files in the folder
file_list = os.listdir(folder_path)

# Initialize an empty list to store the dataframes
dfs = []

# Loop through each file in the folder
for file_name in file_list:
    file_path = os.path.join(folder_path, file_name)
    
    # Check if the file is a pickle file
    if file_name.endswith('.pkl'):
        # Load the dataframe from the pickle file
        df = pd.read_pickle(file_path)
        
        # Append the dataframe to the list
        dfs.append(df)

# Concatenate all the dataframes into a single dataframe
concatenated_df = pd.concat(dfs, ignore_index=True)


In [3]:
concatenated_df.to_pickle(os.path.join(folder_path, 'All_Simulation_Data.pkl'))

In [4]:
df=pd.read_pickle(os.path.join(folder_path, 'All_Simulation_Data.pkl'))
df

,text_file_names,dose_file_names,energy,beam coordinates,angles,beam_spread,ct_name,cropped_dose_name,cropped_geometry_name,shape
0,9.000000-Venous_Phase__5.0__B30f-43947.txt,9.000000-Venous_Phase__5.0__B30f-43947_Run_0000,103.895676,"(16.6, -8.0, -7.7)","(0, 90)",0.164136,9.000000-Venous_Phase__5.0__B30f-43947,ac6ef505-30b2-4f42-8041-a7d152fb26a2,dcbcc7c0-f2b0-4441-b996-d6b7901e8a1a,"(68, 68, 150)"
1,9.000000-Venous_Phase__5.0__B30f-43947.txt,9.000000-Venous_Phase__5.0__B30f-43947_Run_0001,294.097841,"(16.6, -7.0, -7.7)","(0, 90)",0.219522,9.000000-Venous_Phase__5.0__B30f-43947,31fab998-7da9-4134-b2f1-8198f52b42ed,475ceb90-bf60-4450-8eb3-6302bd8fd19e,"(68, 68, 150)"
2,9.000000-Venous_Phase__5.0__B30f-43947.txt,9.000000-Venous_Phase__5.0__B30f-43947_Run_0002,216.962575,"(16.6, -6.0, -7.7)","(0, 90)",0.338667,9.000000-Venous_Phase__5.0__B30f-43947,e1d3caaf-1f05-4491-808f-137eea3aed0f,14753bb0-b91c-4870-9116-c2789f517864,"(68, 68, 150)"
3,9.000000-Venous_Phase__5.0__B30f-43947.txt,9.000000-Venous_Phase__5.0__B30f-43947_Run_0003,378.350985,"(16.6, -5.0, -7.7)","(0, 90)",0.465590,9.000000-Venous_Phase__5.0__B30f-43947,5b5d8ba3-c080-4675-ba8a-1c78be44a309,1ecba431-44c2-44c2-babc-222e6db1fbb2,"(68, 68, 150)"
4,9.000000-Venous_Phase__5.0__B30f-43947.txt,9.000000-Venous_Phase__5.0__B30f-43947_Run_0004,148.526178,"(16.6, -4.0, -7.7)","(0, 90)",0.197513,9.000000-Venous_Phase__5.0__B30f-43947,c2cacf6e-a1a8-4259-a8d3-49fdcc24a66f,5a536360-d522-414a-bc21-5780626319c9,"(68, 68, 150)"
...,...,...,...,...,...,...,...,...,...,...
34255,8.000000-Abdomen__5.0__B30f-52754.txt,8.000000-Abdomen__5.0__B30f-52754_Run_2771,394.153504,"(7.6, 19.2, 6.3)","(270, 0)",0.105562,8.000000-Abdomen__5.0__B30f-52754,7cb01db2-9928-49bf-b5b1-2ed1b9749ad9,f97bfa18-6da9-4956-9d53-c5d46e1e8555,"(68, 68, 150)"
34256,8.000000-Abdomen__5.0__B30f-52754.txt,8.000000-Abdomen__5.0__B30f-52754_Run_2772,145.888801,"(8.6, 19.2, 6.3)","(270, 0)",0.479546,8.000000-Abdomen__5.0__B30f-52754,e3d4809f-643a-444b-999a-b9e95fb8b7e4,73fef386-e364-4c50-812c-f691fc6dc9b5,"(68, 68, 150)"
34257,8.000000-Abdomen__5.0__B30f-52754.txt,8.000000-Abdomen__5.0__B30f-52754_Run_2773,349.860061,"(9.6, 19.2, 6.3)","(270, 0)",0.127348,8.000000-Abdomen__5.0__B30f-52754,891ad8af-32ae-416f-9b8f-853434511ad6,0913e72f-18d0-4a76-b3c4-a594cb615877,"(68, 68, 150)"
34258,8.000000-Abdomen__5.0__B30f-52754.txt,8.000000-Abdomen__5.0__B30f-52754_Run_2774,273.480172,"(10.6, 19.2, 6.3)","(270, 0)",0.457579,8.000000-Abdomen__5.0__B30f-52754,36b89f9c-9181-4d54-9293-fdb6c36be543,48d37b53-641f-492d-9acb-6af302ca5e16,"(68, 68, 150)"


In [15]:
df['beam_spread'].max()

0.4996289153950544

In [14]:
gpu_index = 1

# Set the GPU to be visible and set memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
print('Available GPUs: ', gpus)
mem_growth = tf.config.experimental.get_memory_growth(gpus[gpu_index])
print('Memory growth: ', mem_growth)
if gpus:
    tf.config.experimental.set_visible_devices(gpus[gpu_index], 'GPU')
    tf.config.experimental.set_memory_growth(gpus[gpu_index], True)
    mem_growth = tf.config.experimental.get_memory_growth(gpus[gpu_index])
    print('GPU set to be visible and memory growth set to: ', mem_growth)

'''Initializing Batch size num epochs learning rate and weight decay. This is the same in my model'''
batch_size =  8
num_epochs = 30
learning_rate = 0.001
weight_decay = 0.0001

with open('./hyperparameters_mps.json', 'r') as hfile:
    param = json.load(hfile)

preprocess=False
# Load data files
path = r"/tudelft.net/staff-umbrella/simelectrons/OneGeometryMultipleEnergyMultipleSpread"
data_df=pd.read_pickle(os.path.join(path,'Filtered_Simulation_Data.pkl'))
path_ckpt = './weights/ckpt_multiple_energy/weights.ckpt'
train_split = 0.7
val_split = 0.15
test_split = 0.15
listIDs=data_df['cropped_geometry_name'].tolist()
random.seed(333)
random.shuffle(listIDs)


trainIDs = listIDs[:int(round(train_split*len(listIDs)))]
valIDs = listIDs[int(round(train_split*len(listIDs))):]
testIDs = listIDs[int(round((train_split + val_split)*len(listIDs))):]


test_df=data_df[data_df['cropped_geometry_name'].isin(testIDs)]
test_df.to_pickle(os.path.join(path,'test_picklefile.pkl'))

Available GPUs:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU')]
Memory growth:  True
GPU set to be visible and memory growth set to:  True


In [2]:
path = r"/tudelft.net/staff-umbrella/simelectrons/OneGeometryMultipleEnergyMultipleSpread"
data_df=pd.read_pickle(os.path.join(path,'Filtered_Simulation_Data.pkl'))

In [8]:
filtered_df = data_df[data_df['shape'] == (68, 68,150)]
filtered_df.to_pickle(os.path.join(folder_path, 'Filtered_Simulation_Data.pkl'))

In [4]:
preprocess=True
if preprocess==True:
    scale, errors = get_scaling_factors(path,'GeometryAll','DoseAll', data_df)
    scale_json = json.dumps(scale)
    with open('scale_mpe_mps.json', 'w') as file:
        file.write(scale_json)


100%|██████████| 34224/34224 [2:03:02<00:00,  4.64it/s]  


ValueError: too many values to unpack (expected 2)

In [3]:
import glob
import h5py
import numpy as np
import pydicom
from tqdm import tqdm
import os
import traceback
from concurrent.futures import ProcessPoolExecutor

def process_geometry(args):
    path, ikey, okey, geometry_name, dose_name, e_min, e_max, s_min, s_max = args
    try:
        geometry_path = os.path.join(path, ikey, geometry_name + '.dcm')
        dose_path = os.path.join(path, okey, dose_name + '.dcm')
        geometry = pydicom.dcmread(geometry_path).pixel_array
        dose_temp=pydicom.dcmread(dose_path)
        dose = dose_temp.pixel_array * dose_temp.DoseGridScaling

        x_max, x_min = np.max(geometry), np.min(geometry)
        y_max, y_min = np.max(dose), np.min(dose)
        
        return {'min_max_values': (x_min, x_max, y_min, y_max), 'error_ID': None}
    except Exception as e:
        print(geometry_name)
        return {'min_max_values': None, 'error_ID': geometry_name}

def get_scaling_factors_concurrent(path, ikey, okey, df):
    e_min = df['energy'].min()
    e_max = df['energy'].max()
    s_min, s_max = df['beam_spread'].min() if 'beam_spread' in df.columns else (0, 0), df['beam_spread'].max() if 'beam_spread' in df.columns else (0, 0)


    error_IDs = []

    args_list = [(path, ikey, okey, df.iloc[i]['cropped_geometry_name'], df.iloc[i]['cropped_dose_name'], e_min, e_max, s_min, s_max) for i in range(len(df))]

    results = []
    with ProcessPoolExecutor() as executor:
        results = list(tqdm(executor.map(process_geometry, args_list), total=len(args_list)))

    
    min_value_x=min([x['min_max_values'][0] for x in results])
    max_value_x=max([x['min_max_values'][1] for x in results])
    min_value_y=min([x['min_max_values'][2] for x in results])
    max_value_y=max([x['min_max_values'][3] for x in results])

    scaling_dic = {'x_min': float(min_value_x), 'x_max': float(max_value_x),
                   'y_min': float(min_value_y), 'y_max': float(max_value_y),
                   'e_min': float(e_min), 'e_max': float(e_max),
                   's_min': float(s_min), 's_max': float(s_max)}

    return scaling_dic,results


In [5]:
scaling_dic

{'x_min': -1000.0,
 'x_max': 3070.0,
 'y_min': 0.0,
 'y_max': 0.0007003479182538465,
 'e_min': 100.00407558613455,
 'e_max': 399.99977729117956,
 's_min': 0.10005417315196713,
 's_max': 0.4999857237431612}

In [4]:
scaling_dic,results=get_scaling_factors_concurrent(path,'GeometryAll','DoseAll', data_df)

100%|██████████| 34224/34224 [03:43<00:00, 153.40it/s]


In [7]:
min([len(x['min_max_values']) for x in results])

4

In [64]:
min_values_x=[x['min_max_values'][0] for x in res]
max_values_x=[x['min_max_values'][1] for x in res]
min_values_y=[x['min_max_values'][2] for x in res]
max_values_y=[x['min_max_values'][3] for x in res]  

(array([0]),)

In [91]:
p[29713]

-995

In [98]:
geom_list=os.listdir(os.path.join(path,'GeometryAll'))
geom_list=[x.split('.')[0] for x in geom_list]

In [84]:
data_df=pd.read_pickle(os.path.join(path,'Filtered_Simulation_Data.pkl'))


In [82]:
data_df

,text_file_names,dose_file_names,energy,beam coordinates,angles,beam_spread,ct_name,cropped_dose_name,cropped_geometry_name,shape
0,9.000000-Venous_Phase__5.0__B30f-43947.txt,9.000000-Venous_Phase__5.0__B30f-43947_Run_0000,103.895676,"(16.6, -8.0, -7.7)","(0, 90)",0.164136,9.000000-Venous_Phase__5.0__B30f-43947,ac6ef505-30b2-4f42-8041-a7d152fb26a2,dcbcc7c0-f2b0-4441-b996-d6b7901e8a1a,"(68, 68, 150)"
1,9.000000-Venous_Phase__5.0__B30f-43947.txt,9.000000-Venous_Phase__5.0__B30f-43947_Run_0001,294.097841,"(16.6, -7.0, -7.7)","(0, 90)",0.219522,9.000000-Venous_Phase__5.0__B30f-43947,31fab998-7da9-4134-b2f1-8198f52b42ed,475ceb90-bf60-4450-8eb3-6302bd8fd19e,"(68, 68, 150)"
2,9.000000-Venous_Phase__5.0__B30f-43947.txt,9.000000-Venous_Phase__5.0__B30f-43947_Run_0002,216.962575,"(16.6, -6.0, -7.7)","(0, 90)",0.338667,9.000000-Venous_Phase__5.0__B30f-43947,e1d3caaf-1f05-4491-808f-137eea3aed0f,14753bb0-b91c-4870-9116-c2789f517864,"(68, 68, 150)"
3,9.000000-Venous_Phase__5.0__B30f-43947.txt,9.000000-Venous_Phase__5.0__B30f-43947_Run_0003,378.350985,"(16.6, -5.0, -7.7)","(0, 90)",0.465590,9.000000-Venous_Phase__5.0__B30f-43947,5b5d8ba3-c080-4675-ba8a-1c78be44a309,1ecba431-44c2-44c2-babc-222e6db1fbb2,"(68, 68, 150)"
4,9.000000-Venous_Phase__5.0__B30f-43947.txt,9.000000-Venous_Phase__5.0__B30f-43947_Run_0004,148.526178,"(16.6, -4.0, -7.7)","(0, 90)",0.197513,9.000000-Venous_Phase__5.0__B30f-43947,c2cacf6e-a1a8-4259-a8d3-49fdcc24a66f,5a536360-d522-414a-bc21-5780626319c9,"(68, 68, 150)"
...,...,...,...,...,...,...,...,...,...,...
34255,8.000000-Abdomen__5.0__B30f-52754.txt,8.000000-Abdomen__5.0__B30f-52754_Run_2771,394.153504,"(7.6, 19.2, 6.3)","(270, 0)",0.105562,8.000000-Abdomen__5.0__B30f-52754,7cb01db2-9928-49bf-b5b1-2ed1b9749ad9,f97bfa18-6da9-4956-9d53-c5d46e1e8555,"(68, 68, 150)"
34256,8.000000-Abdomen__5.0__B30f-52754.txt,8.000000-Abdomen__5.0__B30f-52754_Run_2772,145.888801,"(8.6, 19.2, 6.3)","(270, 0)",0.479546,8.000000-Abdomen__5.0__B30f-52754,e3d4809f-643a-444b-999a-b9e95fb8b7e4,73fef386-e364-4c50-812c-f691fc6dc9b5,"(68, 68, 150)"
34257,8.000000-Abdomen__5.0__B30f-52754.txt,8.000000-Abdomen__5.0__B30f-52754_Run_2773,349.860061,"(9.6, 19.2, 6.3)","(270, 0)",0.127348,8.000000-Abdomen__5.0__B30f-52754,891ad8af-32ae-416f-9b8f-853434511ad6,0913e72f-18d0-4a76-b3c4-a594cb615877,"(68, 68, 150)"
34258,8.000000-Abdomen__5.0__B30f-52754.txt,8.000000-Abdomen__5.0__B30f-52754_Run_2774,273.480172,"(10.6, 19.2, 6.3)","(270, 0)",0.457579,8.000000-Abdomen__5.0__B30f-52754,36b89f9c-9181-4d54-9293-fdb6c36be543,48d37b53-641f-492d-9acb-6af302ca5e16,"(68, 68, 150)"


In [6]:
DataGenerator(trainIDs, path, batch_size, data_df, scale).__getitem__(0)

['f92e845b-10d4-4ab2-b448-6bdbf74d3e6c', '934bd72d-dce0-4888-89d8-4e1c2609796f', '8ae4a495-df14-4c0d-a3ea-7de2c175011a', '413b3d79-3eab-47b5-8bb2-6af86224f07e', '851f169c-8273-48e4-bd1d-fb39efe3129a', '61ae8e3a-bf22-458e-8fe1-3d3e304952e0', 'd1cb6031-ba07-446c-bee5-7cf9836fe7a0', '8bd0c0f6-ff76-4e4d-9ce8-fbd354db2f23']


8it [00:02,  3.73it/s]


([array([[[[[0.00000000e+00],
            [0.00000000e+00],
            [4.91400491e-04],
            ...,
            [0.00000000e+00],
            [0.00000000e+00],
            [7.37100737e-04]],
  
           [[2.45700246e-04],
            [0.00000000e+00],
            [0.00000000e+00],
            ...,
            [1.96560197e-03],
            [1.71990172e-03],
            [9.82800983e-04]],
  
           [[1.22850123e-03],
            [1.22850123e-03],
            [1.22850123e-03],
            ...,
            [1.96560197e-03],
            [1.71990172e-03],
            [9.82800983e-04]],
  
           ...,
  
           [[0.00000000e+00],
            [2.45700246e-04],
            [1.47420147e-03],
            ...,
            [1.96560197e-03],
            [1.71990172e-03],
            [4.91400491e-04]],
  
           [[9.82800983e-04],
            [4.91400491e-04],
            [0.00000000e+00],
            ...,
            [0.00000000e+00],
            [0.00000000e+00],
          

In [ ]:

with open('scale_mpe_mps.json', 'r') as file:
    scale_json = file.read()
scale = json.loads(scale_json)


train_gen = DataGenerator(trainIDs, path, batch_size, data_df, scale)
val_gen = DataGenerator(valIDs,  path, batch_size, data_df, scale)


transformer = dota_energies(
    num_tokens=param['num_tokens'],
    input_shape=param['data_shape'],
    projection_dim=param['projection_dim'],
    num_heads=param['num_heads'],
    num_transformers=param['num_transformers'], 
    kernel_size=param['kernel_size'],
    causal=True
)
transformer.summary()

optimizer = LAMB(learning_rate=learning_rate, weight_decay_rate=weight_decay)
transformer.compile(optimizer=optimizer, loss='mse', metrics=[])


# Callbacks.
# Save best model at the end of the epoch.
checkpoint = ModelCheckpoint(
    filepath=path_ckpt,
    save_weights_only=True,
    save_best_only=True,
    monitor='val_loss',
    mode='min')

#Learning rate scheduler. Manually reduce the learning rate.
sel_epochs = [4,8,12,16,20,24,28]
lr_scheduler = LearningRateScheduler(
    lambda epoch, lr: lr*0.5 if epoch in sel_epochs else lr,
    verbose=1)

optimizer.learning_rate.assign(learning_rate)
history = transformer.fit(
    x=train_gen,
    validation_data=val_gen,
    epochs=num_epochs,
    verbose=1,
    callbacks=[checkpoint, lr_scheduler,WandbMetricsLogger(log_freq=10)]
    )

# Save last weights and hyperparameters.
path_last = os.path.join(os.path.dirname(path),'./weights_mpe/weights.ckpt')
transformer.save_weights(path_last)
